In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,MaxPool2D,Conv2D,Flatten,BatchNormalization,Dropout
import os
from PIL import Image
from sklearn.metrics import accuracy_score,precision_score,recall_score
from tensorflow.keras.callbacks import TensorBoard
import PIL

In [2]:
def corrupt_image(dir):
    for root, _, files in os.walk(dir):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                img=Image.open(file_path)
                img.verify()
            except Exception as e:
                print(f"Corrupted image: {file_path}, Error: {e}")

dir = "D:/ImageClassification"
corrupt_image(dir)


Corrupted image: D:/ImageClassification\anaconda_projects\db\project_filebrowser.db, Error: cannot identify image file 'D:\\ImageClassification\\anaconda_projects\\db\\project_filebrowser.db'


In [197]:
datagen=ImageDataGenerator(
    rescale =1.0/255,
    validation_split=0.2,
    horizontal_flip= True,
    rotation_range=20,
    height_shift_range=0.2,
    width_shift_range=0.2,
    zoom_range=0.2
)

In [199]:
train_datagen=datagen.flow_from_directory(
    dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

Found 244 images belonging to 1 classes.


In [201]:
val_datagen=datagen.flow_from_directory(
    dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 61 images belonging to 1 classes.


In [211]:
model=Sequential([
    Conv2D(32,(3,3),input_shape=(150,150,3),activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=(2,2)),
    Conv2D(64,(3,3),activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=(2,2)),
    Conv2D(128,(3,3),activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=(2,2)),
    Flatten(),
    Dense(128,activation='relu'),
    Dropout(0.5),
    Dense(1,activation='sigmoid')])

In [213]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [215]:
ts_board=TensorBoard(log_dir='log',histogram_freq=1)

In [217]:
model.fit( train_datagen, epochs=10, callbacks=[ts_board], validation_data=val_datagen)

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 46s 5s/step - accuracy: 0.8063 - loss: 0.8597 - val_accuracy: 0.9180 - val_loss: 0.1534
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 36s 5s/step - accuracy: 0.9878 - loss: 0.5111 - val_accuracy: 0.0328 - val_loss: 3.4039
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - accuracy: 1.0000 - loss: 7.0088e-21 - val_accuracy: 0.6557 - val_loss: 0.8984
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - accuracy: 0.9991 - loss: 0.0021 - val_accuracy: 0.7541 - val_loss: 0.6457
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - accuracy: 1.0000 - loss: 1.8575e-21 - val_accuracy: 0.5246 - val_loss: 1.7571
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 23s 3s/step - accuracy: 1.0000 - loss: 9.6544e-13 - val_accuracy: 0.2787 - val_loss: 2.7452
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 23s 3s/step - accuracy: 0.9980 - loss: 0.0664 - val_accuracy: 0.0820 - val_loss: 7.2239
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 23s 3s/step - accuracy: 0.9980 - loss: 0.0375 - val_accuracy: 0.2295 - val_loss: 6.6

In [219]:
loss,accuracy=model.evaluate(val_datagen)

2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 0.8823 - loss: 0.7424


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image


def predict_image(model, img_path, target_size=(150, 150)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    

    prediction = model.predict(img_array)
    
    
    if prediction[0][0] > 0.5:
        print(f"Prediction: Class 1 (Positive), Confidence: {prediction[0][0]:.2f}")
    else:
        print(f"Prediction: Class 0 (Negative), Confidence: {1 - prediction[0][0]:.2f}")


image_path = "D:/ImageClassification/test_image.jpg"
predict_image(model, image_path)
